In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
agents_path = "/content/drive/MyDrive/Colab Notebooks/Autonomous/Agents Autonomous"

# Warlords Multi-Agent Toernooi
Dit notebook draait de Warlords-omgeving vanuit de Arcade Learning Environment (ALE) met 4 custom agenten.

Dit notebook is ontworpen om te draaien in Google Colab. Als je hem lokaal draait, moet je mogelijk meer libraries installeren en controleren of hun versies compatibel zijn.

## 1 Installeer libraries

Voer eerst de onderstaande codecell uit om de libraries te installeren.

In [ ]:
# Install the necessary libraries
!pip install pettingzoo[atari]
!pip install "autorom[accept-rom-license]"
!pip install --find-links dist/ --no-cache-dir AutoROM[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.0/552.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.5/852.5 kB 12.5 MB/s eta 0:00:00
  Created wheel for multi_agent_ale_py: filename=multi_agent_ale_py-0.1.11-cp311-cp311-linux_x86_64.whl size=721821 sha256=0c0a47945b880fc4d7234b304ffd3fee01bf232c74c58721b08d3b49122d3020
  Stored in directory: /root/.cache/pip/wheels/1d/81/76/771ec8e34292c8a71dd6c4a52a1c0401f4d93cbfb54e02fce4
Successfully built multi_agent_ale_py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=656de815db68b3c74169ff99ea1e07cd56747433e79474448eb6d0dbcf86205b
  Stored in directory: /root/.cache/pip/wheels/bc/fc/c6/8

**Herstart nu je kernel**. Na het herstarten kun je direct doorgaan met de volgende codecel.

## 2 Importeer libraries en download Atari ROMs

Je krijgt een prompt om de AutoROM-overeenkomst te accepteren. Druk op "Y" wanneer je dit ziet.

In [ ]:
# Start AutoROM

!AutoROM

# Import libraries
from pettingzoo.atari import warlords_v3
from pettingzoo.utils import BaseParallelWrapper
import gymnasium as gym
import numpy as np
from collections import defaultdict, Counter
import importlib
import os
import imageio

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.11/dist-packages/AutoROM/roms
	/usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: y
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/

## 3 Initialiseer agenten

In deze codecel importeren we de AI-agenten om Warlords te spelen. De bestanden met de agentklassen moeten zich in dezelfde map bevinden als dit notebook.

In [21]:
import sys
import os
from collections import defaultdict, Counter

# Voeg het pad toe waar de agent bestanden zich bevinden
agents_path = '/content/drive/MyDrive/Colab Notebooks/Autonomous/Agents Autonomous'
if agents_path not in sys.path:
    sys.path.append(agents_path)

# Nu kun je de agents importeren
from Agent1 import Agent1
from Agent2 import Agent2
from Agent3 import Agent3
from Agent4 import Agent4

# Instantiateer de agents
agent_instances = [
    Agent1(),
    Agent2(),
    Agent3(),
    Agent4()
]

agent_names = ['Agent1', 'Agent2', 'Agent3', 'Agent4']
scores = defaultdict(int)
wins = Counter()


## 4 Speel het spel

In deze sectie spelen de vier agenten Warlords. Aan het einde van elk spel wordt de score bijgehouden. De winnaar is de agent met de hoogste score.

In [22]:
# Create environment
env = warlords_v3.env(render_mode="rgb_array")

# Prepare directory for videos
video_dir = "./warlords_videos"
os.makedirs(video_dir, exist_ok=True)

De volgende codecel speelt het spel.

In [24]:
# Function to run one game and record video
def run_game(game_id):
    env.reset()

    # Map environment agents to their corresponding agent instances
    agent_mapping = {
        env.agents[i]: agent_instances[i]
        for i in range(len(env.agents))
    }

    done = False
    terminated = False
    truncated = False

    frames = []
    rewards = []

    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()

        if termination or truncation:
            action = None
        else:
            # this is where you would insert your policy
            agent_obj = agent_mapping[agent]
            action = agent_obj.act(observation)

        env.step(action)

        # Capture the rendered frame
        frame = env.render()
        frames.append(frame)
    env.close()

    for i, reward in enumerate(rewards):
        scores[agent_names[i]] += reward
    alive_agents = [i for i, r in enumerate(rewards) if r > 0]
    if len(alive_agents) == 1:
        wins[agent_names[alive_agents[0]]] += 1

    # Save video using imageio
    video_path = os.path.join(video_dir, f"game_test{game_id}.mp4")
    imageio.mimsave(video_path, frames, fps=15)

In [25]:
# Run 10 games
for game in range(1):
    print(f"Running game test 2 {game + 1}...")
    run_game(game_id=game)

print("\nFinal Scores:")
for agent in agent_names:
    print(f"{agent}: Total Reward = {scores[agent]}, Wins = {wins[agent]}")

try:
    winner = wins.most_common(1)[0]
    print(f"Winner: {winner[0]} with {winner[1]} wins!")
except IndexError:
    print("No winners found.")

Running game test 2 1...



Final Scores:
Agent1: Total Reward = 0, Wins = 0
Agent2: Total Reward = 0, Wins = 0
Agent3: Total Reward = 0, Wins = 0
Agent4: Total Reward = 0, Wins = 0
No winners found.


In [19]:
# Display download links for videos
import glob
from IPython.display import FileLink, display
video_files = sorted(glob.glob(f"{video_dir}/*.mp4"))
print("\nDownload the recorded games:")
for file in video_files:
    display(FileLink(file))


Download the recorded games:


/content/warlords_videos/game_0.mp4

/content/warlords_videos/game_1.mp4

/content/warlords_videos/game_2.mp4

/content/warlords_videos/game_3.mp4

...

/content/warlords_videos/game_5.mp4

/content/warlords_videos/game_6.mp4

/content/warlords_videos/game_7.mp4

/content/warlords_videos/game_8.mp4

/content/warlords_videos/game_9.mp4

In [20]:
from IPython.display import FileLink, display
import glob

video_files = sorted(glob.glob(f"{video_dir}/*.mp4"))
print("\nDownload the recorded games:")
for file in video_files:
    display(FileLink(file))



Download the recorded games:


/content/warlords_videos/game_0.mp4

/content/warlords_videos/game_1.mp4

/content/warlords_videos/game_2.mp4

/content/warlords_videos/game_3.mp4

...

/content/warlords_videos/game_5.mp4

/content/warlords_videos/game_6.mp4

/content/warlords_videos/game_7.mp4

/content/warlords_videos/game_8.mp4

/content/warlords_videos/game_9.mp4